In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=b5762da0df817859e58b5719c4c71bf6e11a9baa956b96195f5ad0f759ceedfb
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=25f4a158aff27c22cf4db4d911804891e2c24a788766889fe91b9de205ce62d3
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_lables),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
      
    ),
    keras.layers.Dense(10, activation='softmax')#output layer
  ]) 
  

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  

  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                           objective='val_accuracy',
                           max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 13m 22s]
val_accuracy: 0.9041666388511658

Best val_accuracy So Far: 0.9041666388511658
Total elapsed time: 00h 13m 22s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |32                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |64                
conv_2_kernel     |5                 |5                 
dense_1_units     |80                |64                
learning_rate     |0.001             |0.001             

Epoch 1/3
1019/1688 [=================>............] - ETA: 4:11 - loss: 0.4425 - accuracy: 0.8397

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()